In [ ]:
import numpy as np

np.random.seed(5)

from tensorflow.keras.layers import Input, Dense, SimpleRNN
from tensorflow.keras import Model
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import backend as K


In [ ]:
nombres = open("nombres_dinosaurios.txt", "r").read()
nombres = nombres.lower()

In [ ]:
#Definimos el alfabeto para la red neuronal
alfabeto = list(set(nombres))
tam_datos = len(nombres)
tam_alfabetos = len(alfabeto)
print("Datos: ", tam_datos)
print("Alfabeto: ", tam_alfabetos)
print(alfabeto)
car_to_ind = {car: ind for ind, car in enumerate(sorted(alfabeto))}
ind_to_car = {ind: car for ind, car in enumerate(sorted(alfabeto))}
print("Caracter to Index: ", car_to_ind)
print("Index to Caracter: ", ind_to_car)

In [ ]:
#Redes Neuronales
n = 25  #Parametro de Hidden Layers
entrada = Input(shape=(None, tam_alfabetos))  #Tamaño de la palabra variable = None en shape
a0 = Input(shape=(n,))  #Red Recurrente --> Estado Anterior // n que refleja las neuronas que tiene el estado
cel_RNN = SimpleRNN(n, activation="tanh", return_state=True)  #La salida te devuelve el estado actualizado
cap_salida = Dense(tam_alfabetos, activation="softmax")

In [ ]:
#Definimos la recurrencia
hs, _ = cel_RNN(entrada, initial_state=a0)  #Te realimentas con a0 = Estado Inicial
salida = []
salida.append(cap_salida(hs))  #Te alimentas con la salida **Aun no esta hecha la retroalimentacion del estado
modelo = Model([entrada, a0], salida)  #Indicamos que nos estamos retroalimentando con las dos entradas y la salida


In [ ]:
#Creamos el optimizador
opt = SGD(learning_rate=0.0005)
modelo.compile(optimizer=opt, loss="categorical_crossentropy")

#Usar un generador para crear ejemplos de entrenamiento
with open("nombres_dinosaurios.txt") as f:
    ejemplos = f.readlines()
ejemplos = [x.lower().strip() for x in ejemplos]
np.random.shuffle(ejemplos)

In [ ]:
#Pasar los caracteres a indices y despues al  onehot
import tensorflow as tf

class Generator(tf.keras.utils.Sequence):
    def train_generator(self):
        while True:
            ejemplo = ejemplos[np.random.randint(0, len(ejemplos))]
            x = [None] + [car_to_ind[c] for c in ejemplo]
            y = x[1:] + [car_to_ind["\n"]]
            x_onehot = np.zeros((len(x), 1, tam_alfabetos))
            onehot = to_categorical(x[1:], tam_alfabetos).reshape(len(x) - 1, 1, tam_alfabetos)
            x_onehot[1:, :, :] = onehot
            y_onehot = to_categorical(y, tam_alfabetos).reshape(len(x), tam_alfabetos)
            a = np.zeros((len(x), n))
            yield (x_onehot, a), y_onehot
        

In [ ]:
#Entrenamos el modelo 
BATCH_SIZE = 80
N_ITS = 10000
generator = Generator()
for j in range(N_ITS):
    hist = modelo.fit(generator.train_generator(), steps_per_epoch=BATCH_SIZE, epochs=1, verbose=0)
    if j % 1000 == 0:
        print("\n Iteracion: %d, error: %f" % (j, hist.history["loss"][0]) + "\n")

In [ ]:
def name_generator(modelo, car_to_ind, tam_alfabetos, n):
    x = np.zeros((1,1,tam_alfabetos, ))
    a = np.zeros((1,n))
    nombre_generado = ''
    fin_de_linea = '\n'
    car = ''
    count = 0
    while (car != fin_de_linea and count != 50):
        a,_ =cel_RNN(K.constant(x), initial_state = K.constant(a))
        y = cap_salida(a)
        pred = K.eval(y)    #Predicciones del modelo
        ix = np.random.choice(list(range(tam_alfabetos)), p=pred.ravel()) #Escogemos un elemento
        car = ind_to_car[ix] # El elemento lo convertimos a caracter
        nombre_generado += car
        x = to_categorical(ix, tam_alfabetos).reshape(1,1,tam_alfabetos)
        a = K.eval(a)
        count += 1

        if count == 50:
            car = fin_de_linea
    print(nombre_generado)
        

In [ ]:
for i in range (100):
    name_generator(modelo, car_to_ind, tam_alfabetos, n)